In [38]:
import requests

In [39]:
url = "https://cluster.elasticsearch.dataesr.ovh/bsso-publications/_search"
header = {'Authorization' : open("../../.env_es_bsso", "r").read().strip()}

In [40]:
# Onglet Les Archives

In [41]:
## Quelle est la dynamique d’ouverture de la santé parmi les archives ?

In [43]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { "oa_details." + year + ".oa_host_type": "repository"   }}
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0, 'doc_count': 28239},
   {'key': 2019.0, 'doc_count': 25598},
   {'key': 2018.0, 'doc_count': 23889},
   {'key': 2017.0, 'doc_count': 22624},
   {'key': 2016.0, 'doc_count': 20711},
   {'key': 2015.0, 'doc_count': 18420},
   {'key': 2014.0, 'doc_count': 14868},
   {'key': 2013.0, 'doc_count': 10051},
   {'key': 2021.0, 'doc_count': 3331},
   {'key': 2012.0, 'doc_count': 303}]}}

In [44]:
QUESTION : pourquoi oa_details n'est pas un simple tableau d'objects avec un champ "millésime" (ou autre) afin de pouvoir faire une aggrégation sur ce champ ? 

SyntaxError: invalid syntax (<ipython-input-44-8262e4317fdb>, line 1)

In [45]:
## Quelles archives ouvertes sont les plus utilisées en santé ?

In [46]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { "oa_details." + year + ".oa_host_type": "repository"   }}
      ]
    }
  },
  "aggs": {
    "by_repository": {
      "terms": {
        "field": "oa_details." + year + ".repositories.keyword",
        "size": 200,
        "missing": "MISSING"
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_repository': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 17806,
  'buckets': [{'key': 'www.ncbi.nlm.nih.gov', 'doc_count': 120214},
   {'key': 'europepmc.org', 'doc_count': 103256},
   {'key': 'HAL', 'doc_count': 55321},
   {'key': 'pdfs.semanticscholar.org', 'doc_count': 32544},
   {'key': 'arxiv.org', 'doc_count': 8266},
   {'key': 'discovery.ucl.ac.uk', 'doc_count': 2789},
   {'key': 'www.biorxiv.org', 'doc_count': 2309},
   {'key': 'www.pure.ed.ac.uk', 'doc_count': 2219},
   {'key': 'www.research.ed.ac.uk', 'doc_count': 2205},
   {'key': 'spiral.imperial.ac.uk', 'doc_count': 1856},
   {'key': 'escholarship.org', 'doc_count': 1779},
   {'key': 'eprints.whiterose.ac.uk', 'doc_count': 1734},
   {'key': 'univoak.eu', 'doc_count': 1723},
   {'key': 'digital.csic.es', 'doc_count': 1671},
   {'key': 'ora.ox.ac.uk', 'doc_count': 1601},
   {'key': 'archimer.ifremer.fr', 'doc_count': 1538},
   {'key': 'www.zora.uzh.ch', 'doc_count': 1476},
   {'key': 'helda.helsinki.fi',

In [47]:
QUESTION : On s'arrête à combien de repositories ? Est-ce que certains repos ne devraient pas être mergés (Je n'ai pas trouvé d'exemple en ce sens) ?

SyntaxError: invalid syntax (<ipython-input-47-ee0c88a3a8e0>, line 1)

In [48]:
ERROR : Pour certaines publications, il manque le champ "oa_details.year.repositories.keyword". ex: https://cluster.elasticsearch.dataesr.ovh/bsso-publications/_search?q=_id:1573. `{'key': 'MISSING', 'doc_count': 257}`

SyntaxError: invalid syntax (<ipython-input-48-2a52f2cba37b>, line 1)

In [49]:
## Quelle est la dynamique de dépôt par archive ouverte en santé ?

In [50]:
year = "2021Q1"
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { "oa_details." + year + ".oa_host_type": "repository"   }}
      ]
    }
  },
  "aggs": {
    "by_discipline": {
      "terms": {
        "field": "oa_details." + year + ".repositories.keyword"
      },
      "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "publication_year"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_discipline': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 101602,
  'buckets': [{'key': 'www.ncbi.nlm.nih.gov',
    'doc_count': 120214,
    'by_publication_year': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 2020.0, 'doc_count': 21565},
      {'key': 2019.0, 'doc_count': 18189},
      {'key': 2018.0, 'doc_count': 17003},
      {'key': 2017.0, 'doc_count': 15852},
      {'key': 2016.0, 'doc_count': 14517},
      {'key': 2015.0, 'doc_count': 13130},
      {'key': 2014.0, 'doc_count': 10366},
      {'key': 2013.0, 'doc_count': 6782},
      {'key': 2021.0, 'doc_count': 2572},
      {'key': 2012.0, 'doc_count': 238}]}},
   {'key': 'europepmc.org',
    'doc_count': 103256,
    'by_publication_year': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 2019.0, 'doc_count': 16814},
      {'key': 2018.0, 'doc_count': 16657},
      {'key': 2017.0, 'doc_count': 15657},
      {'key': 2016.0, 'd

In [52]:
Question : Cette requête est basée sur le millésime "2021Q1". Est-ce bien la façon dont ce graphe a été créé ?

SyntaxError: invalid syntax (<ipython-input-52-e8d80379cc89>, line 1)

In [53]:
## Quelle place occupe HAL dans la dynamique des archives ouvertes en santé ?

In [80]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { "oa_details." + year + ".oa_host_type": "repository"   }}
      ]
    }
  },
  "aggs": {
    "by_discipline": {
      "terms": {
        "field": "oa_details." + year + ".repositories.keyword"
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_discipline': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 101602,
  'buckets': [{'key': 'www.ncbi.nlm.nih.gov', 'doc_count': 120214},
   {'key': 'europepmc.org', 'doc_count': 103256},
   {'key': 'HAL', 'doc_count': 55321},
   {'key': 'pdfs.semanticscholar.org', 'doc_count': 32544},
   {'key': 'arxiv.org', 'doc_count': 8266},
   {'key': 'discovery.ucl.ac.uk', 'doc_count': 2789},
   {'key': 'www.biorxiv.org', 'doc_count': 2309},
   {'key': 'www.pure.ed.ac.uk', 'doc_count': 2219},
   {'key': 'www.research.ed.ac.uk', 'doc_count': 2205},
   {'key': 'spiral.imperial.ac.uk', 'doc_count': 1856}]}}

In [ ]:
IDEA : Should post process to group / sum all repositories other than 'HAL'